## Graph Modeling
When modeling you data in a graph database, it's generally beneficial to follow a structure that allows you to effectively query and leverage the relationshipps within the data.
1. Understand the domain and define specific use cases (questions) for the application
2. Develop the initial graph data model
    - Model the nodes (entities)
    - Model the relationships between nodes
3. Test the use cases against the initial data model.
4. Create the graph (instance model) with test data using Cypher.
5. Test the use cases, including performance against the graph.
6. Refactor (improve) the graph data model due to a change in the key use cases or for performance reasons.
7. Implement the refactring on the graph and retest using Cypher.

## Domain Knowledge
1. Identify the stakeholders and developers of the application.
2. With the stakeholders and developers:
    - Describe the application in detail.
    - Identify the users of the applicaiton (people, systems).
    - Agree upon the use cases for the application.
    - Rank the importance of the use cases.

## Two types of Model:
1. Data Model:A data model is an abstract model that organizes data elements and standardizes how they relate to one another and to the properties of real-world entities.

2. Instance Model: An instance model, on the other hand, refers to the actual data or database instance that adheres to the structure of the **Data Model**.

We Create an **Instance Model** after creating **Data Model** because we need to confirm that our data model satisfies the use cases of our application.

**Abstract vs. Concrete**: A **Data Model** is abstract, focusing on the blueprint and design, whereas an **Instance Model** is concrete, representing actual data at a particular time.

## Model Relationships:
1. The relationship name should be all captials.
2. The start and end label must have a label associated to them.

## QUERY Example:
1. What role did a person play in the movie?
    - Retrieve the name of the person.
    - Follow the ACTED_IN relationship to movies
    - Filter the movie by its title.
    - Return the role from the ACTED_IN relationship.



## Basic Example Questions - 
1. What was the revenue of Apple in the first quarter of 2024?
2. What are the risk factors associated with Adobe?
3. How were the net sales of Amazon in 2024 compared to 2023?
4. What was the percentage difference in total revenues of Netflix between 2024 and 2023, which year seen more sales.?

**BASED ON THESE BASIC QUESTIONS IT WOULD BE BETTER TO MODEL GRAPH IN THE FOLLLOWING STRUCTURE:**

NODES:
- Filing: accession, form
- ParentChunk: attributes - content, sub_heading_dict, is_table, chunk_id
- ChildChunk: attributes - content, chunk_id
- Company: attributes - name
- Heading: attributes - name

RELATIONSHIPS:
- HAS_CHILD
- FILED:
- HAS_SECTION
- PART_OF: attributes - filing_date

PATTERN
- (Company)-[:FILED]->(Filing)
- (Filing)-[:HAS_SECTION]->(Heading)
- (ParentChunk)-[:HAS_CHILD]->(ChildChunk)
- (ParentChunk)-[:PART_OF]->(Heading)
- (Filing)-[:NEXT]->(Filing)

NOTE - To avoid passing very large parent sections to the LLM, sibling relationships make it easy to retrieve a limited context window (top-K related children) without loading the entire parent chunk.

## Create The Graph

We will use **Domain Graph** only to create Graph from our databset. Because of Computational and Storage Costs we will avoid using chunks to extract nodes and relationships, we might explore that in future.<br>

https://graphrag.com/reference/knowledge-graph/domain-graph/<br>
https://graphrag.com/reference/knowledge-graph/lexical-graph/<br>

The Structure of the graph will be in Parent-Child Lexical Graph pattern and the retrieval pattern will also be using same Parent-Child Retrieval.<br>
https://graphrag.com/reference/graphrag/parent-child-retriever/


In [1]:
import os
from dotenv import load_dotenv
import time
from mistralai import Mistral
from neo4j import GraphDatabase
import pandas as pd
from pathlib import Path
import json
from openai import AzureOpenAI
import pprint

load_dotenv()


True

## Load Data

In [2]:
#Load the data
TENK_TENQ_DATA = os.getenv("OUT_ALL")
chunks = {}
with open(TENK_TENQ_DATA,'r') as file:
    chunks = json.load(file)


In [3]:
# rows = []
# for file in chunks:
#     for chunk in file["file_chunks"]:
#         for id, child_chunk in chunk["Chunks"]["child_chunks"].items():
#             rows.append({
#                 "id": id,
#                 "chunk_content": child_chunk,
#                 "mistral_embedding": []
#             })

# df_embed = pd.DataFrame(rows)

In [4]:
# mistral_embedding_cache = {}
# with open('mistral_embedding_cache.json','r') as file:
#     mistral_embedding_cache = json.load(file)

## Embeddings

### 1. Mistral Embeddings

In [5]:
# # Create a function to get the embedding
# # It return 1024 dimensional vector
# def generate_embedding(llm, chunk_list):
#     response = llm.embeddings.create(
#     inputs=chunk_list,
#     model="mistral-embed"
#     )
#     return response.data

In [6]:
# # Create OpenAI object
# llm  = Mistral(api_key=os.getenv("MISTRAL_API"))

In [7]:
# for start in range(0, len(df_embed), 100):
#     batch = df_embed.iloc[start:start+100]
#     texts = batch["chunk_content"].to_list()

#     to_embed = []
#     map_idx = []

#     # Find which ones need embeddings
#     for idx, text in enumerate(texts):
#         if text not in embedding_cache:
#             to_embed.append(text)
#             map_idx.append(idx)

#     # Call Mistral only for new chunks
#     if to_embed:
#         time.sleep(1)
#         new_embeddings = generate_embedding(llm, to_embed)
#         for idx, emb in zip(map_idx, new_embeddings):
#             mistral_embedding_cache[texts[idx]] = emb.embedding

#     # Assign embeddings back in the correct row order
#     df_embed.loc[start:start+len(texts)-1, "mistral_embedding"] = [
#         mistral_embedding_cache[t] for t in texts
#     ]


In [8]:
## # Save the embeddings to csv
## df_embed.to_csv("child_embeddings.csv",index=False)
## df_embed.to_parquet("child_embeddings.parquet",index=False)
## with open("mistral_embedding_cache.json",'w', encoding="utf-8") as file:
##     json.dump(mistral_embedding_cache,file,indent=2)

In [9]:
# EMBED_DF = pd.read_parquet("child_embeddings.parquet")
# EMBED_DF.head()

### 2. OpenAI Embeddings

In [10]:
# df_embed = pd.read_parquet("child_embeddings.parquet")
# df_embed["openai_embedding"] = [[] for _ in range(len(df_embed))]
# df_embed.head()

In [11]:
# model = AzureOpenAI(
#     api_version="2024-12-01-preview",
#     azure_endpoint=os.getenv("OPENAI_EMBED_ENDPOINT"),
#     api_key=os.getenv("OPENAI_EMBED_API")
# )

In [12]:
# def generate_openai_embed(model, chunks):
#     response = model.embeddings.create(
#     input=chunks,
#     model=os.getenv("OPENAI_EMBED_DEPLOYMENT")
#     )
#     return response.data


In [13]:
# openai_embedding_cache = {}

In [14]:
# for start in range(0, len(df_embed), 350):
#     batch = df_embed.iloc[start:start+350]
#     texts = batch["chunk_content"].to_list()


#     to_embed = []
#     map_idx = []

#     # Find which ones need embeddings
#     for idx, text in enumerate(texts):
#         if text not in openai_embedding_cache:
#             to_embed.append(text)
#             map_idx.append(idx)

#     # Call Mistral only for new chunks
#     if to_embed:
#         new_embeddings = generate_openai_embed(model, to_embed)
#         for idx, emb in zip(map_idx, new_embeddings):
#             openai_embedding_cache[texts[idx]] = emb.embedding
#         time.sleep(60)

#     # Assign embeddings back in the correct row order
#     # Create the list of embeddings
#     embeddings = [openai_embedding_cache[t] for t in texts]

#     # Wrap in a Series to force Pandas to treat each embedding as a single object
#     # and match the index of the slice you are targetting
#     target_index = df_embed.index[start : start + len(texts)]

#     df_embed.loc[target_index, "openai_embedding"] = pd.Series(
#         embeddings, 
#         index=target_index, 
#         dtype=object
#     )


In [15]:
## # Save the embeddings to csv
## df_embed.to_csv("child_embeddings_openai.csv",index=False)
## df_embed.to_parquet("child_embeddings_openai.parquet",index=False)
## with open("openai_embedding_cache.json",'w', encoding="utf-8") as file:
##     json.dump(openai_embedding_cache,file,indent=2)

## Preparing data for Graph

In [16]:
embed_path = Path("C:/Users/paymo/Downloads/QuantiGence/data/embeddings/child_embeddings_openai.parquet")
EMBED_DF = pd.read_parquet(embed_path)
EMBED_DF.head()

,id,chunk_content,mistral_embedding,openai_embedding
0,1_0,"The Company designs, manufactures and markets ...","[-0.04815673828125, 0.0364990234375, 0.0552062...","[0.010563179850578308, 0.011085673235356808, 0..."
1,2_0,iPhone iPhone® is the Company’s line of smartp...,"[-0.0309600830078125, 0.044891357421875, 0.059...","[0.008778193034231663, -0.004154624883085489, ..."
2,3_0,Mac® is the Company’s line of personal compute...,"[-0.02239990234375, 0.033111572265625, 0.05377...","[-0.0024118726141750813, 0.000555747770704329,..."
3,4_0,"Wearables includes smartwatches, wireless head...","[-0.0255889892578125, 0.015594482421875, 0.042...","[0.01224506739526987, 0.003041462041437626, -0..."
4,5_0,The Company’s advertising services include thi...,"[-0.04876708984375, 0.028564453125, 0.03784179...","[0.02215135656297207, 0.017149832099676132, 0...."


In [17]:
#get embeddings from csv
def get_embeddings(child_chunks):
    embeddings = {}
    for child_id, _ in child_chunks.items():
        row = EMBED_DF.loc[EMBED_DF['id'] == child_id, "openai_embedding"]
        if row.empty:
            raise ValueError(f"Missing embedding for {child_id}")
        vec = row.values[0]
        embeddings[child_id] = [float(x) for x in vec]

    return embeddings

# Create a function to get the course data
def get_course_data(chunk_obj):
    data = {}

    metadata = chunk_obj["Metadata"]
    chunks = chunk_obj["Chunks"]
    chunk_id = chunk_obj["ID"]
    
    data['parent_chunk_content'] = chunks["parent_chunk"]
    data["parent_chunk_id"] = chunk_id
    data["child_chunks"] = chunks["child_chunks"]
    data['filing_date'] = metadata["filing_date"] 
    data["heading_name"] = metadata.get("item_heading","Not Found")
    data["form_type"] = metadata["form"]
    data["company_name"] = metadata["company"]
    data["is_table"] = metadata["is_table"]
    data["accession_no"] = metadata["accession_no"]
    data["sub_headings"] = data["sub_headings"] = json.dumps(metadata.get("sub_headings", {}))
    data['embeddings'] = get_embeddings(data["child_chunks"])
    return data

In [18]:
# Connect to Neo4j
def connect_neo4j():
    try:
        driver = GraphDatabase.driver(
            os.getenv('NEO4J_URI'),
            auth=(
                os.getenv('NEO4J_USERNAME'),
                os.getenv('NEO4J_PASSWORD')
            )
        )
        driver.verify_connectivity()
        print("Successfully connected to the database")
        return driver
    except Exception as e:
        print("Connection was unsuccessful due to: ",e)
        return None


In [19]:
# driver = connect_neo4j()
# with driver.session() as session:
#     # Delete the entire graph
#     session.run("MATCH (n) DETACH DELETE n")

# # Close the driver connection when done
# driver.close()

In [20]:

def create_nodes_and_relationships_batch(tx, batch):
    result = tx.run("""
    UNWIND $batch AS data

    // 1. Company
    MERGE (company:Company {name: data.company_name})

    // 2. Filing 
    MERGE (filing:Filing {accession: data.accession_no})
    SET filing.form = data.form_type, filing.date = date(data.filing_date)

    MERGE (company)-[:FILED]->(filing)

    // 3. Unique Heading (The Fix)
    // By including accession_no, this node is now unique to THIS filing.
    MERGE (heading:Heading {
        name: data.heading_name, 
        accession: data.accession_no
    })
    MERGE (filing)-[:HAS_SECTION]->(heading)

    // 4. ParentChunk (Linked to the unique heading)
    MERGE (parent:ParentChunk {chunk_id: data.parent_chunk_id})
    SET parent.content = data.parent_chunk_content,
        parent.sub_heading_dict = data.sub_headings,
        parent.is_table = data.is_table

    MERGE (heading)-[:HAS_CONTENT]->(parent)

    WITH parent, data

    // 5. ChildChunks (Rule 4)
    UNWIND keys(data.child_chunks) AS child_id
    MERGE (child:ChildChunk {chunk_id: child_id})
    SET child.content = data.child_chunks[child_id]

    MERGE (parent)-[:HAS_CHILD]->(child)

    WITH child, data, child_id
    CALL db.create.setNodeVectorProperty(
        child,
        "embedding",
        data.embeddings[child_id]
    )
    """, batch=batch)
    return result.consume()

In [21]:
# Function to initialize vector index
def initialize_vector_index(tx):
    tx.run("""
        CREATE VECTOR INDEX childchunks IF NOT EXISTS
        FOR (child:ChildChunk)
        ON child.embedding
        OPTIONS {indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'
        }}
    """)

In [22]:
BATCH_SIZE = 10   # you can raise to 20 later
batch_id = 0
driver = connect_neo4j()

# Batched ingestion
with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
    buffer = []

    for file in chunks:
        for chunk_obj in file["file_chunks"]:
            buffer.append(get_course_data(chunk_obj))

            if len(buffer) == BATCH_SIZE:
                try:
                    print(f"Writing batch {batch_id} (size={len(buffer)})")
                    summary = session.execute_write(create_nodes_and_relationships_batch, buffer)
                    print("Created {nodes_created} and {relationships_created} nodes in {time} ms.".format(
                        nodes_created=summary.counters.nodes_created,
                        relationships_created = summary.counters.relationships_created,
                        time=summary.result_available_after
                    ))
                    print(f"Batch {batch_id} done")
                    batch_id += 1

                    buffer.clear()
                except Exception as e:
                    print("The session did not run due to: ",e)

        print(f"Done for File - {file['ticker']} with date - {file['filing_date']}")

    # flush remaining
    if buffer:
        print(f"Writing batch {batch_id} (size={len(buffer)})")
        summary = session.execute_write(create_nodes_and_relationships_batch, buffer)
        print("Created {nodes_created} and {relationships_created} relationships in {time} ms.".format(
            nodes_created=summary.counters.nodes_created,
            relationships_created = summary.counters.relationships_created,
            time=summary.result_available_after
        ))
        print(f"Batch {batch_id} done")
        driver.close()
print("SUCCESS!! The DATABASE IS CREATED")

# Create vector index once
with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
    session.execute_write(initialize_vector_index)
    print("Vector Index has been Set")
    driver.close()

Successfully connected to the database
Writing batch 0 (size=10)
Created 0 and 0 nodes in 12 ms.
Batch 0 done
Writing batch 1 (size=10)
Created 0 and 0 nodes in 19 ms.
Batch 1 done
Writing batch 2 (size=10)
Created 0 and 0 nodes in 9 ms.
Batch 2 done
Writing batch 3 (size=10)
Created 0 and 0 nodes in 8 ms.
Batch 3 done
Writing batch 4 (size=10)
Created 0 and 0 nodes in 8 ms.
Batch 4 done
Writing batch 5 (size=10)
Created 0 and 0 nodes in 9 ms.
Batch 5 done
Writing batch 6 (size=10)
Created 0 and 0 nodes in 8 ms.
Batch 6 done
Writing batch 7 (size=10)
Created 0 and 0 nodes in 8 ms.
Batch 7 done
Writing batch 8 (size=10)
Created 0 and 0 nodes in 9 ms.
Batch 8 done
Writing batch 9 (size=10)
Created 0 and 0 nodes in 8 ms.
Batch 9 done
Writing batch 10 (size=10)
Created 0 and 0 nodes in 7 ms.
Batch 10 done
Writing batch 11 (size=10)
Created 0 and 0 nodes in 8 ms.
Batch 11 done
Writing batch 12 (size=10)
Created 0 and 0 nodes in 7 ms.
Batch 12 done
Writing batch 13 (size=10)
Created 0 and 0 

C:\Users\paymo\AppData\Local\Temp\ipykernel_17868\4047658830.py:45: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:


Vector Index has been Set


## POST PROCESSING THE GRAPH FOR FIANCIAL TIME SERIES

1. Create a Date property on "Filing" nodes.
2. Convert string to neo4j date format in filing and PART_OF relationships.
3. Add quarter labels to filings like (Q1,Q2,Q3).
4. Create NEXT relationships among filings of 10-k and 10-q separately.

In [23]:
def create_10q_period(tx):
    query = """
       
    // Step 1 — Assign sequential quarters per company + year
    MATCH (c:Company)-[:FILED]->(f:Filing {form: "10-Q"})
    WITH c, f, f.date.year AS year
    ORDER BY c.name, year, f.date
    WITH c, year, collect(f) AS filings
    UNWIND range(0, size(filings)-1) AS i
    WITH filings[i] AS f, i, year
    SET f.period = "Q" + toString(i + 1) + "_" + toString(year);
    """

    results = tx.run(query)
    return results.consume()

def create_10k_period(tx):
    query = """
    // Step 2 — Set FY period for 10-K
    MATCH (f:Filing {form: "10-K"})
    SET f.period = "FY_" + toString(f.date.year);
    """
    results = tx.run(query)
    return results.consume()

def create_next_rel(tx):
    query = """
    // Step 3 — Build NEXT relationships per company + form
    MATCH (c:Company)-[:FILED]->(f:Filing)
    WITH c, f.form AS form, f
    ORDER BY c.name, form, f.date
    WITH c, form, collect(f) AS filings
    UNWIND range(0, size(filings)-2) AS i
    WITH filings[i] AS f1, filings[i+1] AS f2
    MERGE (f1)-[:NEXT]->(f2);
    """
    results = tx.run(query)
    return results.consume()


In [24]:
# Post processing the graph with new attributes and realtionships
driver = connect_neo4j()
try:
    with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
        summary = session.execute_write(create_10q_period)
        print("Created {relationships_created} relationships and set {properties_set} properties in {time} ms.".format(
            relationships_created=summary.counters.relationships_created,
            properties_set=summary.counters.properties_set,
            time=summary.result_available_after
        ))
    driver.close()
except Exception as e:
    print("Session did not connect: ",e)

# Post processing the graph with new attributes and realtionships
driver = connect_neo4j()
try:
    with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
        summary = session.execute_write(create_10k_period)
        print("Created {relationships_created} relationships and set {properties_set} properties in {time} ms.".format(
            relationships_created=summary.counters.relationships_created,
            properties_set=summary.counters.properties_set,
            time=summary.result_available_after
        ))
    driver.close()
except Exception as e:
    print("Session did not connect: ",e)

# Post processing the graph with NEXT realtionships
driver = connect_neo4j()
try:
    with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
        summary = session.execute_write(create_next_rel)
        print("Created {relationships_created} relationships and set {properties_set} properties in {time} ms.".format(
            relationships_created=summary.counters.relationships_created,
            properties_set=summary.counters.properties_set,
            time=summary.result_available_after
        ))
    driver.close()
except Exception as e:
    print("Session did not connect: ",e)

Successfully connected to the database
Created 0 relationships and set 65 properties in 192 ms.
Successfully connected to the database
Created 0 relationships and set 23 properties in 38 ms.
Successfully connected to the database
Created 66 relationships and set 0 properties in 120 ms.


## Testing and Verificaton

In [25]:
def verify_graph():
    query = """
    MATCH (n)
    WITH count(n) AS totalNodes, sum(size(keys(n))) AS totalAttributes
    MATCH (f:Filing)
    WITH totalNodes, totalAttributes, count(f) AS totalFiling
    MATCH (c:Company)
    WITH totalNodes, totalAttributes, totalFiling, count(c) AS totalCompanies
    MATCH (h:Heading)
    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, count(h) AS totalHeadings
    MATCH (p:ParentChunk)
    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, totalHeadings, count(p) AS totalParentChunks
    MATCH (ch:ChildChunk)
    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, totalHeadings, totalParentChunks, count(ch) AS totalChildChunks
    MATCH ()-[has_child:HAS_CHILD]->()
    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, totalHeadings, totalParentChunks, totalChildChunks,
        count(has_child) AS totalChildRelationships
    MATCH ()-[part_of:PART_OF]->()
    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, totalHeadings, totalParentChunks, totalChildChunks,
        totalChildRelationships, count(part_of) AS totalPartOfRelationships
    MATCH ()-[has_sec:HAS_SECTION]->()
    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, totalHeadings, totalParentChunks, totalChildChunks,
        totalChildRelationships, totalPartOfRelationships, count(has_sec) AS totalHasSectionRelationships
    MATCH ()-[filed:FILED]->()
    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, totalHeadings, totalParentChunks, totalChildChunks,
        totalChildRelationships, totalPartOfRelationships, totalHasSectionRelationships, count(filed) AS totalFiledRelationships
    RETURN totalNodes, totalAttributes, totalFiling, totalCompanies, totalHeadings, totalParentChunks, totalChildChunks,
        totalChildRelationships, totalPartOfRelationships, totalHasSectionRelationships, totalFiledRelationships

    """
    return query

In [26]:
# Post processing the graph with new attributes and realtionships
driver = connect_neo4j()

records, summary, keys = driver.execute_query(verify_graph(),database=os.getenv("NEO4J_DATABASE", "neo4j"))

driver.close()
        

Successfully connected to the database


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: PART_OF)} {position: line: 17, column: 23, offset: 911} for query: '\n    MATCH (n)\n    WITH count(n) AS totalNodes, sum(size(keys(n))) AS totalAttributes\n    MATCH (f:Filing)\n    WITH totalNodes, totalAttributes, count(f) AS totalFiling\n    MATCH (c:Company)\n    WITH totalNodes, totalAttributes, totalFiling, count(c) AS totalCompanies\n    MATCH (h:Heading)\n    WITH totalNodes, totalAttributes, totalFiling, totalCompanies, count(h) AS totalHeadings\n    MATCH (p:ParentChunk)\n    WITH totalNodes, totalAttributes, totalFiling, t

In [27]:
#Printing all the records
for record in records:
    pprint.pprint(record.data())

In [28]:
def test_graph():
    query = """
    //  Pick Apple and order filings
    MATCH (c:Company {name:"Apple Inc."})-[:FILED]->(f:Filing)
    WITH c, f ORDER BY f.date

    //  Check NEXT chaining
    OPTIONAL MATCH (f)-[:NEXT]->(n)
    WITH c, f, n

    //  Find Risk Factors sections using substring
    MATCH (f)-[:HAS_SECTION]->(h)
    WHERE toLower(h.name) CONTAINS "risk factors"
    WITH c, f, n, h

    //  Get ParentChunks
    MATCH (p:ParentChunk)-[:PART_OF]->(h)
    WITH c, f, n, h, p

    // Get ChildChunks
    MATCH (p)-[:HAS_CHILD]->(ch)
    WITH c.name AS company, f.period AS period, f.form AS form,
        h.name AS section, coalesce(n.period, "END") AS next_period,
        p.chunk_id AS parent_id, ch.chunk_id AS child_id

    RETURN company, period, form, section, next_period, parent_id, child_id
    ORDER BY period

    """
    return query

In [29]:
# Post processing the graph with new attributes and realtionships
driver = connect_neo4j()

records, summary, keys = driver.execute_query(test_graph(),database=os.getenv("NEO4J_DATABASE", "neo4j"))

driver.close()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: PART_OF)} {position: line: 16, column: 29, offset: 423} for query: '\n    //  Pick Apple and order filings\n    MATCH (c:Company {name:"Apple Inc."})-[:FILED]->(f:Filing)\n    WITH c, f ORDER BY f.date\n\n    //  Check NEXT chaining\n    OPTIONAL MATCH (f)-[:NEXT]->(n)\n    WITH c, f, n\n\n    //  Find Risk Factors sections using substring\n    MATCH (f)-[:HAS_SECTION]->(h)\n    WHERE toLower(h.name) CONTAINS "risk factors"\n    WITH c, f, n, h\n\n    //  Get ParentChunks\n    MATCH (p:ParentChunk)-[:PART_OF]->(h)\n    WITH c, f, n, h

Successfully connected to the database


## ADDING EARNINGS CALL TRANSCRIPT

## INTEGRATED GRAPH MODEL


NODES:<br>
ParentChunk - attr: content, chunk_id<br>
ChildChunk - attr: content, chunk_id
speaker - name<br>
title - title_name<br>
transcript - attr: year, quarter<br>

RELETIONSHIPS:<br>
HAS_SPEECH<br>
HAS_TITLE<br>
HAS_CHILD<br>
HAS_CHUNK<br>
HAS_TRANSCRIPT<br>
NEXT<br>

PATTERNS:<br>
company --HAS_TRANSCRIPT--> transcript<br>
transcript --HAS_CHUNK--> ParentChunk<br>
ParentChunk --HAS_CHILD--> ChildChunk<br>
speaker --HAS_SPEECH--> content<br>
speaker --HAS_TITLE--> title<br>
transcript --NEXT--> transcript

## Get Data

In [30]:
transcript_chunks = None
chunk_path = Path("C:/Users/paymo/Downloads/QuantiGence/Chunks/all_transcripts.json")
embed_path = Path("C:/Users/paymo/Downloads/QuantiGence/data/embeddings/child_embeddings_transcript.parquet")
with open(chunk_path,'r') as file:
    transcript_chunks = json.load(file)

EMBED_DF_TRASNCRIPT = pd.read_parquet(embed_path)

In [31]:
#get embeddings from csv
def get_embeddings(child_chunks):
    embeddings = {}
    for child_id, _ in child_chunks.items():
        row = EMBED_DF_TRASNCRIPT.loc[EMBED_DF_TRASNCRIPT['id'] == child_id, "openai_embedding"]
        if row.empty:
            raise ValueError(f"Missing embedding for {child_id}")
        vec = row.values[0]
        embeddings[child_id] = [float(x) for x in vec]

    return embeddings

# Create a function to get the course data
def get_course_data(chunk_obj):
    data = {}

    metadata = chunk_obj["Metadata"]
    parent_chunk = chunk_obj["ParentChunk"]
    child_chunks = chunk_obj["ChildChunks"]
    
    data['parent_chunk_content'] = parent_chunk["chunk"]
    data["parent_chunk_id"] = parent_chunk['ID']
    data["child_chunks"] = child_chunks
    data['period'] = metadata['quarter'] + '_' + metadata["year"]
    data["company_name"] = metadata["company"]
    data["speaker_name"] = metadata["speaker"]
    data["speaker_title"] = metadata["title"]
    data['embeddings'] = get_embeddings(data["child_chunks"])
    return data

In [32]:
def create_transcript_nodes_and_relationships_batch(tx, batch):
    result = tx.run("""
    UNWIND $batch AS data

    // Match company
    MATCH (company:Company {name: data.company_name})

    // Match filing using SAME company + period tag
    MATCH (company)-[:FILED]->(f:Filing)
    WHERE f.form = "10-Q"
      AND f.period = data.period   // e.g. Q1_2024, Q2_2024

    // Transcript node (period-aligned with filing)
    MERGE (t:Transcript {
        company: data.company_name,
        period: data.period,
        id: data.company_name + '_' + data.period
    })

    MERGE (company)-[:HAS_TRANSCRIPT]->(t)
    MERGE (t)-[:ASSOCIATED_WITH]->(f)

    // Speaker + Title
    MERGE (s:Speaker {name: data.speaker_name})
    MERGE (title:Title {title_name: data.speaker_title})
    MERGE (s)-[:HAS_TITLE]->(title)

    // Parent chunk
    MERGE (p:TranscriptParentChunk {chunk_id: data.parent_chunk_id})
    SET p.content = data.parent_chunk_content

    MERGE (t)-[:HAS_CHUNK]->(p)
    MERGE (s)-[:HAS_SPEECH]->(p)

    WITH t, p, data

    // Child chunks
    UNWIND keys(data.child_chunks) AS child_id
    MERGE (tc:TranscriptChildChunk {chunk_id: child_id})
    SET tc.content = data.child_chunks[child_id]

    MERGE (p)-[:HAS_CHILD]->(tc)
    WITH t, tc, data, child_id

    // Embeddings
    CALL db.create.setNodeVectorProperty(
        tc,
        "embedding",
        data.embeddings[child_id]
    )
    """, batch=batch)

    return result.consume()


In [33]:
## initialize vector index only once

# Function to initialize vector index
def initialize_vector_index(tx):
    tx.run("""
        CREATE VECTOR INDEX tc IF NOT EXISTS
        FOR (tc:TranscriptChildChunk)
        ON tc.embedding
        OPTIONS {indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'
        }}
    """)

In [34]:
BATCH_SIZE = 10   # you can raise to 20 later
batch_id = 0
driver = connect_neo4j()
# Create vector index once
with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
    session.execute_write(initialize_vector_index)

# Batched ingestion
with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
    buffer = []

    for file in transcript_chunks:
        for chunk_obj in file["transcript"]:
            buffer.append(get_course_data(chunk_obj))

            if len(buffer) == BATCH_SIZE:
                try:
                    print(f"Writing batch {batch_id} (size={len(buffer)})")
                    summary = session.execute_write(create_transcript_nodes_and_relationships_batch, buffer)
                    print("Created {nodes_created} and {relationships_created} nodes in {time} ms.".format(
                        nodes_created=summary.counters.nodes_created,
                        relationships_created = summary.counters.relationships_created,
                        time=summary.result_available_after
                    ))
                    print(f"Batch {batch_id} done")
                    batch_id += 1

                    buffer.clear()
                except Exception as e:
                    print("The session did not run due to: ",e)

        print(f"Done for File - {file['ticker']} with year - {file['year']} and Quarter {file['quarter']}")

    # flush remaining
    if buffer:
        print(f"Writing batch {batch_id} (size={len(buffer)})")
        summary = session.execute_write(create_transcript_nodes_and_relationships_batch, buffer)
        print("Created {nodes_created} nodes and {relationships_created} relationships in {time} ms.".format(
            nodes_created=summary.counters.nodes_created,
            relationships_created = summary.counters.relationships_created,
            time=summary.result_available_after
        ))
        print(f"Batch {batch_id} done")

print("SUCCESS!! The DATABASE IS CREATED")
driver.close()

Successfully connected to the database
Writing batch 0 (size=10)
Created 46 and 52 nodes in 254 ms.
Batch 0 done
Writing batch 1 (size=10)
Created 21 and 31 nodes in 10 ms.
Batch 1 done
Writing batch 2 (size=10)
Created 22 and 32 nodes in 8 ms.
Batch 2 done
Writing batch 3 (size=10)
Created 21 and 31 nodes in 8 ms.
Batch 3 done
Writing batch 4 (size=10)
Created 22 and 32 nodes in 5 ms.
Batch 4 done
Writing batch 5 (size=10)
Created 21 and 31 nodes in 148 ms.
Batch 5 done
Done for File - AAPL with year - 2024 and Quarter Q1
Writing batch 6 (size=10)
Created 34 and 45 nodes in 6 ms.
Batch 6 done
Writing batch 7 (size=10)
Created 20 and 30 nodes in 5 ms.
Batch 7 done
Writing batch 8 (size=10)
Created 20 and 30 nodes in 5 ms.
Batch 8 done
Writing batch 9 (size=10)
Created 20 and 30 nodes in 4 ms.
Batch 9 done
Writing batch 10 (size=10)
Created 20 and 30 nodes in 6 ms.
Batch 10 done
Writing batch 11 (size=10)
Created 21 and 31 nodes in 5 ms.
Batch 11 done
Writing batch 12 (size=10)
Created 

## Building NEXT Relatiosnhip betwween transcripts

In [35]:
def build_next_rel(tx):
    result = tx.run("""MATCH (c:Company)-[:HAS_TRANSCRIPT]->(t:Transcript)
    WITH c, t
    ORDER BY c.name, t.period
    WITH c, collect(t) AS transcripts
    UNWIND range(0, size(transcripts)-2) AS i
    WITH transcripts[i] AS t1, transcripts[i+1] AS t2
    MERGE (t1)-[:NEXT]->(t2)""")
    return result.consume()

In [36]:
# Post processing the graph with new attributes and realtionships
driver = connect_neo4j()
try:
    with driver.session(database=os.getenv("NEO4J_DATABASE", "neo4j")) as session:
        summary = session.execute_write(build_next_rel)
        print("Created {relationships_created} relationships and set {properties_set} properties in {time} ms.".format(
            relationships_created=summary.counters.relationships_created,
            properties_set=summary.counters.properties_set,
            time=summary.result_available_after
        ))

except Exception as e:
    print("Session did not connect: ",e)

Successfully connected to the database
Created 54 relationships and set 0 properties in 71 ms.


In [37]:
# Post processing the graph with new attributes and realtionships

def verify():
    query = """
    MATCH (c:Company)-[:HAS_TRANSCRIPT]->(t:Transcript)
    RETURN c.name, count(t) AS transcript_nodes
    ORDER BY transcript_nodes;
        
    """
    return query
driver = connect_neo4j()

records, summary, keys = driver.execute_query(verify(),database=os.getenv("NEO4J_DATABASE", "neo4j"))

driver.close()

Successfully connected to the database


In [38]:
import pprint
pprint.pprint(records)

[<Record c.name='ORACLE CORP' transcript_nodes=5>,
 <Record c.name='Apple Inc.' transcript_nodes=6>,
 <Record c.name='MICROSOFT CORP' transcript_nodes=6>,
 <Record c.name='Alphabet Inc.' transcript_nodes=6>,
 <Record c.name='AMAZON COM INC.' transcript_nodes=6>,
 <Record c.name='Meta Platforms, Inc.' transcript_nodes=6>,
 <Record c.name='NVIDIA CORP' transcript_nodes=6>,
 <Record c.name='Tesla, Inc.' transcript_nodes=6>,
 <Record c.name='Salesforce, Inc.' transcript_nodes=6>,
 <Record c.name='NETFLIX INC' transcript_nodes=6>,
 <Record c.name='ADOBE INC.' transcript_nodes=6>]


In [39]:
dict_count = {}

import pprint
for file in transcript_chunks:
    if not dict_count.get(file["ticker"],False):
        dict_count[file["ticker"]] = [file["quarter"]+'_'+file["year"]]
    else:
        dict_count[file["ticker"]].append(file["quarter"]+'_'+file["year"])

dict_count

{'AAPL': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'MSFT': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'GOOGL': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'AMZN': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'META': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'NVDA': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'TSLA': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'ORCL': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'CRM': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'NFLX': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025'],
 'ADBE': ['Q1_2024', 'Q2_2024', 'Q3_2024', 'Q1_2025', 'Q2_2025', 'Q3_2025']}

In [40]:
# Post processing the graph with new attributes and realtionships

def verify_filing():
    query = """
    MATCH (c:Company)-[:FILED]->(f:Filing {form:"10-Q"})
    RETURN c.name, collect(f.period) AS filing_periods    
    """
    return query
driver = connect_neo4j()

records, summary, keys = driver.execute_query(verify_filing(),database=os.getenv("NEO4J_DATABASE", "neo4j"))

driver.close()

Successfully connected to the database


In [41]:
pprint.pprint(records)

[<Record c.name='Apple Inc.' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='MICROSOFT CORP' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='Alphabet Inc.' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='AMAZON COM INC.' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='Meta Platforms, Inc.' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='NVIDIA CORP' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='Tesla, Inc.' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='ORACLE CORP' filing_periods=['Q2_2025', 'Q1_2025', 'Q3_2024', 'Q2_2024', 'Q1_2024']>,
 <Record c.name='Salesforce, Inc.' filing_periods=['Q3_2025', 'Q2_2025', 'Q1_2025'

# GRAPH VERIFICATION

In [42]:
driver = connect_neo4j()

Successfully connected to the database


In [43]:
create_nodes_and_relationships_batch
create_transcript_nodes_and_relationships_batch

<function __main__.create_transcript_nodes_and_relationships_batch(tx, batch)>

In [44]:
# Post processing the graph with new attributes and realtionships

def verify_filing():
    query = """
    MATCH (c:Company)-[:HAS_TRANSCRIPT]->(t:Transcript)-[:HAS_CHUNK]->(p:TranscriptParentChunk)-[:HAS_CHILD]->(tc:TranscriptChildChunk)
    WHERE p.chunk_id = 13987
    RETURN DISTINCT c.name,p.chunk_id, t.period,tc.chunk_id
    """
    return query
driver = connect_neo4j()

records, summary, keys = driver.execute_query(verify_filing(),database=os.getenv("NEO4J_DATABASE", "neo4j"))

driver.close()

Successfully connected to the database


In [45]:
records

[<Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_0'>,
 <Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_7'>,
 <Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_5'>,
 <Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_6'>,
 <Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_3'>,
 <Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_4'>,
 <Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_1'>,
 <Record c.name='Alphabet Inc.' p.chunk_id=13987 t.period='Q3_2025' tc.chunk_id='13987_2'>]

In [46]:
# Verify SEC

def verify_filing():
    query = """
   MATCH (c:Company)-[:FILED]->(f:Filing)-[:HAS_SECTION]->(h:Heading)-[:HAS_CONTENT]->(p:ParentChunk)-[:HAS_CHILD]->(ch:ChildChunk)
    WHERE p.chunk_id = 5402
    RETURN DISTINCT c.name, p.chunk_id, ch.chunk_id,f.period;
    """
    return query
driver = connect_neo4j()

records, summary, keys = driver.execute_query(verify_filing(),database=os.getenv("NEO4J_DATABASE", "neo4j"))

driver.close()

Successfully connected to the database


In [47]:
records

[<Record c.name='Apple Inc.' p.chunk_id=5402 ch.chunk_id='5402_2' f.period='Q3_2025'>,
 <Record c.name='Apple Inc.' p.chunk_id=5402 ch.chunk_id='5402_0' f.period='Q3_2025'>,
 <Record c.name='Apple Inc.' p.chunk_id=5402 ch.chunk_id='5402_1' f.period='Q3_2025'>]